In [1]:
import sys
import os

import numpy as np
from tqdm.notebook import tqdm

# add parent dir to be able to import utils file
parent_dir = os.path.abspath('../../')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from duqling_py.duqling_r import DuqlingR
from duqling_py.duqling import Duqling as DuqlingPy

In [2]:
duq_r  = DuqlingR()
duq_py = DuqlingPy()

In [3]:
def lhs_array(n: int, d: int, ranges:np.ndarray, seed: int | None = None) -> np.ndarray:
    rng = np.random.default_rng(seed)
    u = rng.random((n, d))
    perms = np.column_stack([rng.permutation(n) for _ in range(d)])
    samples = (perms + u) / n
    samples = ranges[:, 0] + samples * (ranges[:, 1] - ranges[:, 0])
    return samples

In [4]:
NUM_SAMPLES = 100

stochastic_piston_kwargs = dict( 
    Ta_generate = lambda:np.random.beta(10, 15), 
    P0_generate = lambda:np.random.uniform(0.49, 0.51)
)

for fname in tqdm(duq_r.quack().fname):
    kwargs = stochastic_piston_kwargs if fname == 'stochastic_piston' else dict()
    func_info_r  = duq_r .quack(fname)
    func_info_py = duq_py.quack(fname)

    # not all functions in the R duqling repo have 'stochastic' as a quack key
    func_info_r  = {k:v for k,v in func_info_r .items() if k!='stochastic'}
    func_info_py = {k:v for k,v in func_info_py.items() if k!='stochastic'}

    try:
        assert func_info_r.keys() == func_info_py.keys()
    except:
        print(f'{fname} has inconsistent quack keys across language implemetations')

    for k,v_r in func_info_r.items():
        v_py = func_info_py[k]

        try: 
            assert type(v_r) == type(v_py)
        except AssertionError:
            print(f'{fname} has inconsistent {k} types across language implementations')

        try:
            if isinstance(v_r, np.ndarray):
                assert (np.isclose(v_r, v_py)).all()
            else:
                assert v_r == v_py
        except AssertionError:
            print(f'{fname} has inconsistent {k} values across language implementations')

    input_dim   = func_info_py['input_dim']
    input_range = func_info_py['input_range']

    try:
        assert input_dim == input_range.shape[0]
    except:
        print(f'{fname} input ranges inconsistent with input dimension')
    
    X = lhs_array(NUM_SAMPLES, input_dim, input_range)
    if fname == 'dts_sirs':
        X[:,1] = np.minimum(X[:,1], 1-X[:,0])

    try:
        y_r = duq_r .batch_duq(X=X, f=fname, **kwargs)
        y_p = duq_py.batch_duq(X=X, f=fname, **kwargs)
        try:
            assert y_r.shape == y_p.shape
            try:
                assert np.isclose(y_r, y_p, equal_nan=True).all()
            except AssertionError:
                print(f'{fname} has inconsistent function outputs across language implementations')
        except AssertionError:
            print(f'{fname} has inconsistent array output shapes across language implementations')
    except:
        print(f'An error occured while testing {fname}')

  0%|          | 0/61 [00:00<?, ?it/s]

ocean_circ has inconsistent function outputs across language implementations
dts_sirs has inconsistent function outputs across language implementations


/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/numpy/_core/numeric.py:353: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')
/Users/reidmorris/Desktop/duqling/duqling_py/functions/dts_sirs.py:27: RuntimeWarning: overflow encountered in scalar add
  S[t] = S[t-1] + births - deaths[0] - infect + resusc
/Users/reidmorris/Desktop/duqling/duqling_py/functions/dts_sirs.py:27: RuntimeWarning: overflow encountered in scalar subtract
  S[t] = S[t-1] + births - deaths[0] - infect + resusc
/Users/reidmorris/Desktop/duqling/duqling_py/functions/wingweight.py:84: RuntimeWarning: invalid value encountered in scalar power
  fact4 = (100*tc / np.cos(LamCaps))**(-0.3)


## Summary of issues

- `ocean_circ` and `dts_sirs` are stochastic, so the failed equivalence tests aren't surprising